Imports

In [1]:
import tensorflow as tf
import csv
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow_addons as tfa
import csv

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

'''
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
'''

/home/harvey/tf-env/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


"\nfrom tensorflow.keras.mixed_precision import experimental as mixed_precision\npolicy = mixed_precision.Policy('mixed_float16')\nmixed_precision.set_policy(policy)\n"

Load all the tfrecords.

In [2]:
records = []
from os import walk
for (dirpath, dirnames, filenames) in walk("records/"):
    for f in filenames:
        if "train" in f:
            records.append("records/{}".format(f))

print(records)

['records/train10-2071.tfrec', 'records/train12-2071.tfrec', 'records/train11-2071.tfrec', 'records/train06-2071.tfrec', 'records/train03-2071.tfrec', 'records/train02-2071.tfrec', 'records/train05-2071.tfrec', 'records/train13-2071.tfrec', 'records/train01-2071.tfrec', 'records/train15-2061.tfrec', 'records/train14-2071.tfrec', 'records/train07-2071.tfrec', 'records/train00-2071.tfrec', 'records/train08-2071.tfrec', 'records/train04-2071.tfrec', 'records/train09-2071.tfrec']


In [3]:
dataset = tf.data.TFRecordDataset(filenames = records)

In [4]:
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.int64),
}

Store the amount of images for each patient in patient_image_count.
<br>
Store a mappting of image name -> patient id in image_to_patient

In [5]:
patient_image_count = {}
image_to_patient = {}

with open("train.csv", "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    headings = next(reader)
    for im in reader:
        name = im[0]
        patient_id = im[1]
        if patient_id not in patient_image_count:
            patient_image_count[patient_id] = 0
        patient_image_count[patient_id] += 1
        image_to_patient[name] = patient_id
        #print(i, cimages[i]["Target"], ctarget)

In [6]:
IMG_SIZE = 32

longest = max(patient_image_count.values())
print("Longest sequence is: {}".format(longest))

Longest sequence is: 115


Initialize each patient's structure as follows:<br>
In the 'Targets' array we will store a (longest,) numpy array for each patient containing the targets for each image.<br>
In the 'Masks' array we will store a (longest,) numpy array for each patient containing it's mask.<br>
In the 'Images' array we will store a (longest X IMG_SIZE X IMG_SIZE X 3) numpy array for each patient containing the images.<br>
In the 'Written' array we will store an index of how many examples have been written for each patient, This will be useful in the next step.<br>
In the 'Patient_Order' array we will store a mapping of patient_name -> position in other array, This will be useful in the next step.

In [7]:
targets = []
masks = []
images = []

written = []
patient_order = {}

i = 0

for patient in patient_image_count.keys():
    target_arr = np.full((longest,), -1)
    mask_arr = np.ones((longest,))
    image_arr = np.zeros((longest, IMG_SIZE, IMG_SIZE, 3))
    
    targets.append(target_arr)
    masks.append(mask_arr)
    images.append(image_arr)
    
    written.append(0)
    patient_order[patient] = i
    i += 1

Iterate over the dataset and do the following for each example:
<br>
Parse it's image and get the images name.<br>
Find out which patient the image is for.<br>
Find out where in the order this patient is.<br>
Find how many images have been written for this patient.<br>
Increment this patients written value.<br>
Write the image to the correct position in this patients image array.<br>
Write the target to the correct position in this patients target array.<br>
Set the mask value to 0 in the correct position in this patients mask array.

In [8]:
for image in dataset:
    # Parse image.
    example = tf.io.parse_single_example(image, feature_description)
    img = tf.io.decode_image(example['image'], expand_animations = False, channels=3)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))

    # Get image name.
    name = example['image_name'].numpy().decode("utf_8")

    # Get image target.
    target = example['target'].numpy()
    
    # Work out which patient this belongs to.
    patient = image_to_patient[name]
    
    # Find out patient's index.
    patient_index = patient_order[patient]
    
    # Find how many images have been written to this patient.
    written_index = written[patient_index]
    
    # Write image.
    images[patient_index][written_index, :, :, :] = img
    
    # Write target.
    targets[patient_index][written_index] = target
    
    # Set mask to 0.
    masks[patient_index][written_index] = 0
    
    # Increment written value.
    written[patient_index] += 1

In [9]:
images[0][0].dtype

dtype('float64')

Split into train/val sets

In [10]:
total = len(targets)
#for i in iter(alldata):
#    total += 1
#print("{} images in total.".format(total))

In [11]:
train = 0.66
val = 0.33

train_num = int(total*train)
val_num = int(total*val)

In [12]:
for i in range(len(masks)):
    masks[i] = np.reshape(masks[i], (1, 1, -1))

In [13]:
train_targets = np.array(targets[0:train_num])
val_targets = np.array(targets[train_num:])

train_masks = np.array(masks[0:train_num])
val_masks = np.array(masks[train_num:])

train_images = np.array(images[0:train_num])
val_images = np.array(images[train_num:])

In [14]:
print(train_targets.shape)
print(val_targets.shape)

print(train_images.shape)
print(val_images.shape)

print(train_masks.shape)
print(val_masks.shape)

(1356, 115)
(700, 115)
(1356, 115, 32, 32, 3)
(700, 115, 32, 32, 3)
(1356, 1, 1, 115)
(700, 1, 1, 115)


Define helper functions for model, Most of these are heavily inspired by( Some just copy-pasted ) from the official tensorflow tutorial on Transformer networks.

In [121]:
'''
def scaled_dot_product_attention(q, k, v, mask):
    # Matrix multiply the querys by the keys
    qk = tf.matmul(q, k, transpose_b=True, name="queryXkey")
    
    # Get the dimension of the keys and scale the attention by it
    key_dim = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention = tf.cast(qk, tf.float32)/tf.math.sqrt(key_dim)
    
    # Use the mask to set the masked images( The padding ) to a very small number. Softmax will zero these out.
    canceller = mask*-(10**9)
    
    scaled_attention += canceller
    
    # Softmax the attention weights
    attention_weights = tf.nn.softmax(scaled_attention, axis=-1, name="softmax_attention")
    
    # Multiply the values by the weights
    out = tf.matmul(attention_weights, tf.cast(v, tf.float32), name="attention_out")
    
    return out, attention_weights
    '''
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [122]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
        
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [123]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [124]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [125]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
          
  def call(self, x, training, mask):
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)


In [126]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization

In [127]:
def create_image_embedding_net(filters, out_dim, name):
    inp = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    conv = Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), activation="relu")(inp)
    maxpool = MaxPool2D(pool_size=(5, 5))(conv)
    flatten = Flatten()(maxpool)
    out = Dense(out_dim)(flatten)
    submodel = tf.keras.Model(name=name, inputs=[inp], outputs=[out])
    return submodel

In [128]:
def create_output_net(vec_shape, dimension, name):
    inp = tf.keras.Input(shape=(vec_shape,))
    internal = Dense(dimension, activation="relu")(inp)
    internal = BatchNormalization()(internal)
    output = Dense(1, activation="sigmoid")(internal)
    submodel = tf.keras.Model(name=name, inputs=[inp], outputs=[output])
    return submodel

In [129]:
embedding_filters = 128

transformer_layers = 2
transformer_dimension = 512
num_heads = 8

transformer_ff_dimension = 2048

out_hidden_dimension = 512

In [145]:
num_images = longest

# Create input for images and masks.
images_in = tf.keras.Input(name="Images_In", shape=(num_images, IMG_SIZE, IMG_SIZE, 3))
mask_in  = tf.keras.Input(name="Mask_In", shape=(1, 1, num_images))

# Create ConvNet to compute query, key and value vectors
embedding_net = create_image_embedding_net(embedding_filters, transformer_dimension, "embedding_net")

embeddings = []

for i in range(num_images):
    img = images_in[:, i, :, :, :]
    embeddings.append(embedding_net(img))

embeddings = tf.stack(embeddings, axis=1)
print("EMBEDDINGS SHAPE:")
print(embeddings.shape)

transformer = Encoder(num_layers=transformer_layers, d_model=transformer_dimension, num_heads=num_heads, dff=transformer_ff_dimension)

new_image_vectors = transformer(embeddings, True, mask_in)
print(new_image_vectors.shape)
print(new_image_vectors[:, 0, :].shape)

output_net = create_output_net(transformer_dimension, out_hidden_dimension, "output_net")

outputs = []

for i in range(num_images):
    img_vec = new_image_vectors[:, i, :]
    cout = output_net(img_vec)
    outputs.append(cout)

print(outputs[0])
output = tf.stack(outputs, axis=-1)[:, 0, :]
print(output.shape)

model = tf.keras.Model(inputs=[images_in, mask_in], outputs=output)

EMBEDDINGS SHAPE:
(None, 115, 512)
(None, 115, 512)
(None, 512)
Tensor("output_net/dense_149/Sigmoid:0", shape=(None, 1), dtype=float32)
(None, 115)


In [ ]:
test_embed_net = create_image_embedding_net(embedding_filters, transformer_dimension, "embedding_net")

In [ ]:
embeddings = []
for i in range(10):
    embeddings.append(test_embed_net(np.array([images[0][i]])))

In [ ]:
embeddings_stack = tf.stack(embeddings, axis=1)

Precision - 
How many of the melanoma that we said are bad were actually bad?
Recall -
How many of the bad melanoma did we catch?

In [ ]:
tf.keras.utils.plot_model(model)

In [51]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Images_In (InputLayer)          [(None, 115, 32, 32, 0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_232 ( [(None, 32, 32, 3)]  0           Images_In[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_strided_slice_233 ( [(None, 32, 32, 3)]  0           Images_In[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_strided_slice_234 ( [(None, 32, 32, 3)]  0           Images_In[0][0]                  
_______________________________________________________________________________________

In [ ]:
'''
def create_loss_func():
    
    bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    def loss_function(real, pred):
        mask = tf.math.logical_not(tf.math.equal(real, -1))
        ctarget = tf.math.equal(real, 1)
        loss_ = bce(ctarget, pred)

        mask = tf.cast(mask, dtype=loss_.dtype)
        print("LMASK")
        print(mask)
        loss_ *= mask
        print("LOSS-MASKED")
        print(loss_)

        return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
    
    return loss_function
'''

In [131]:
def masked_log_loss():
    def loss_function(real, pred):
        epsilon = 9*(10**-5)
        
        mask = tf.math.logical_not(tf.math.equal(real, -1))
        ctarget = tf.math.equal(real, 1)

        #print("PREDICTIONS")
        #print(pred)
        #print("ADDED EPSILON")
        #print(pred+epsilon)
        
        positive_loss = tf.math.log(pred+epsilon)
        negative_loss = tf.math.log((1-pred)+epsilon)
        
        #print("POSITIVE LOSS")
        #print(positive_loss)
        #print("NEGATIVE LOSS")
        #print(negative_loss)
        
        ctarget = tf.cast(ctarget, dtype=positive_loss.dtype)
        
        positive_loss = positive_loss*ctarget
        negative_loss = negative_loss*(1-ctarget)
        
        loss_ = positive_loss+negative_loss
        loss_ *= -1
        
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ = loss_*mask
        
        #loss_ = tf.reduce_sum(loss_)
        
        return loss_
    return loss_function

In [146]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
          loss=masked_log_loss(),
          metrics=["AUC"])

In [ ]:
stopper = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    patience=3,
    verbose=1,
    mode="max",
    restore_best_weights=True
)

#hist = model.fit([train_images, train_masks], train_targets, validation_data=([val_images, val_masks], val_targets),epochs=100, callbacks=[stopper], batch_size=1)

hist = model.fit([train_images[0:32], train_masks[0:32]], train_targets[0:32], validation_data=([val_images, val_masks], val_targets), epochs=100, batch_size=16)

Epoch 1/100
2/2 [==============================] - 5s 3s/step - loss: 0.2584 - auc: 0.6236 - val_loss: 0.0182 - val_auc: 0.7470
Epoch 2/100
2/2 [==============================] - 1s 748ms/step - loss: 0.2135 - auc: 0.7170 - val_loss: 0.2072 - val_auc: 0.7468
Epoch 3/100
2/2 [==============================] - 2s 752ms/step - loss: 0.1976 - auc: 0.8077 - val_loss: 0.0115 - val_auc: 0.9966
Epoch 4/100
2/2 [==============================] - 2s 751ms/step - loss: 0.0814 - auc: 0.8611 - val_loss: 0.0124 - val_auc: 0.9902
Epoch 5/100
2/2 [==============================] - 2s 751ms/step - loss: 0.0827 - auc: 0.8486 - val_loss: 0.0122 - val_auc: 0.9983
Epoch 6/100
2/2 [==============================] - 2s 754ms/step - loss: 0.0658 - auc: 0.8590 - val_loss: 0.0150 - val_auc: 0.9963
Epoch 7/100
2/2 [==============================] - 2s 753ms/step - loss: 0.0538 - auc: 0.8557 - val_loss: 0.0179 - val_auc: 0.9949
Epoch 8/100
2/2 [==============================] - 1s 750ms/step - loss: 0.0529 - auc:

In [ ]:
np.array([train_images[123]]).shape
np.array([train_masks[123]]).shape

inp_imgs = np.array(train_images[123:125])
inp_masks = np.array(train_masks[123:125])

print(inp_imgs.shape)
print(inp_masks.shape)

pred = model([inp_imgs, inp_masks])
#model([np.random.random_sample((1, 115, 32, 32, 3)), np.random.random_sample((1, 115))])
targets = train_targets[123:125]

print("TARGETS")
print(targets)

print("PREDICTIONS")
print(pred)

In [138]:
with tf.GradientTape() as tape:
    #idx = 123
    idx = 3
    
    inp_imgs = np.array([train_images[idx]])
    inp_masks = np.array([train_masks[idx]])
    
    pred = model([inp_imgs, inp_masks])
    targets = train_targets[idx]
    
    loss_func = masked_log_loss()
    loss = loss_func(targets, pred)
    print(loss.numpy())

[[5.4183750e+00 5.8383574e-03 5.7123424e-03 5.3755022e-03 3.6833088e-03
  4.6125771e-03 4.5039002e-03 2.7939333e-03 3.3401540e-03 4.9950453e-03
  5.6851273e-03 3.9912136e-03 6.7494684e-03 3.2018367e-03 3.8001530e-03
  6.6992426e-03 3.2004016e-03 3.7729898e-03 5.0327252e-03 6.0050422e-03
  4.2160689e-03 5.3683114e-03 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000

In [46]:
loss_func = masked_log_loss()

In [47]:
pred = pred.numpy()

In [139]:
print("TARGETS")
print(targets)

print("PREDICTIONS ")
print(pred)

print("PREDICTIONS ROUNDED")
print(np.round(pred))

print("LOSS")
print(loss_func(targets, pred).numpy())

TARGETS
[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
PREDICTIONS 
tf.Tensor(
[[0.00434435 0.00591137 0.00578605 0.0054511  0.00376655 0.00469194
  0.00458377 0.00288003 0.00342461 0.00507261 0.00575902 0.00407325
  0.00681674 0.00328674 0.00388297 0.00676686 0.00328527 0.00385586
  0.00511008 0.00607704 0.00429718 0.00544392 0.00431865 0.00554444
  0.00442902 0.00325339 0.0036963  0.00725182 0.00684295 0.00459226
  0.00853777 0.00549021 0.00341797 0.00422258 0.00322568 0.00536036
  0.00742019 0.00461209 0.00552038 0.00479626 0.00547661 0.00523791
  0.00554298 0.00375619 0.00396322 0.00530478 0.00541471 0.00360179
  0.00617535 0.004556   0.00788334 0.00420581 0.00437685 0.00577084
  0.0

In [140]:
gradients = tape.gradient(loss, model.trainable_variables)

In [141]:
np.count_nonzero(np.round(gradients[0].numpy()))

0

In [ ]:
pred = pred.numpy()
rpred = pred.reshape(targets.shape)

In [ ]:
targets

In [ ]:
pred[0][3] = 1
pred

In [ ]:
bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

print("Original")
print(bce(targets, pred).numpy())

print("Reshape")
print(bce(targets, rpred).numpy())

In [ ]:
bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

targets[0:2]

In [ ]:
test_records = []

for (dirpath, dirnames, filenames) in walk("records/"):
    for f in filenames:
        if "test" in f:
            test_records.append("records/{}".format(f))

print(test_records)

In [ ]:
feature_description_test = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string)
}

In [ ]:
test_dataset = tf.data.TFRecordDataset(filenames = test_records)

In [ ]:
test_names = []

for inp in test_dataset:
    example = tf.io.parse_single_example(inp, feature_description_test)
    test_names.append(example['image_name'].numpy().decode("utf_8"))

In [ ]:
test_names[0]

In [ ]:
@tf.function
def toInput(inp):
    example = tf.io.parse_single_example(inp, feature_description_test)
        
    img = tf.io.decode_image(example['image'], expand_animations = False, channels=3)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))

    return img

In [ ]:
test_batches = test_dataset.map(toInput)

In [ ]:
test_batches = test_batches.batch(BATCH_SIZE)

In [ ]:
predictions = model.predict(test_batches)

In [ ]:
import csv

i = 0

with open("submission.csv", "w") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["image_name", "target"])
    for p in predictions:
        prounded = f"{p[0]:.9f}"
        writer.writerow([test_names[i], prounded])
        i += 1